In [84]:
import pandas as pd
import numpy as np
import pandas as pd
import numpy as np
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import cross_validate
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV
from imblearn.over_sampling import SMOTE
from collections import Counter
from sklearn.naive_bayes import MultinomialNB 
from sklearn.metrics import recall_score
from sklearn.metrics import precision_score
from sklearn.metrics import f1_score
from sklearn.model_selection import train_test_split

In [93]:
############Leemos el CSV######################
df = pd.read_csv('poker-hand-tiny.csv')
print("Contenido de data set: ", df.shape)
print("Cantidad por clase:\n", df.Clase.value_counts())

Contenido de data set:  (300000, 11)
Cantidad por clase:
 0    150289
1    126833
2     14395
3      6238
4      1156
5       598
6       423
7        62
8         4
9         2
Name: Clase, dtype: int64


In [86]:
############Limpiamos el CSV###################
for i in range(5,10):
    #Creamos lista con los indices a eliminar
    idmemor = df[df['Clase']== i].index
    #Eliminamos los indices del csv
    df = df.drop(idmemor)


In [87]:
#Define cada dataset
y_train = df.iloc[:,10]
x_train = df.iloc[:,0:10]

In [89]:
###################### CREACIÓN DATOS SINTÉTICOS #####################
smote = SMOTE()
X_train, Y_train = smote.fit_sample(x_train, y_train)
print(Counter(y_train), Counter(Y_train))

Counter({0: 150289, 1: 126833, 2: 14395, 3: 6238, 4: 1156}) Counter({0: 150289, 1: 150289, 2: 150289, 3: 150289, 4: 150289})


In [94]:
############# ÁRBOL DE DECISIÓN #################
arboldecision = DecisionTreeClassifier()
parametros = {'criterion': ['gini', 'entropy'],'splitter':['best', 'random'],'max_depth': [1,4,7,8,9,10]}

In [95]:
############# VALIDACIÓN CRUZADA - Grid Search ##############
clf = GridSearchCV(arboldecision, parametros, cv = 3)
clf.fit(X_train, Y_train)
clf.cv_results_
df = pd.DataFrame(clf.cv_results_)
print(df[['param_criterion', 'param_max_depth', 'param_splitter', 'mean_test_score', 'rank_test_score' ]])

   param_criterion param_max_depth param_splitter  mean_test_score  \
0             gini               1           best         0.226203   
1             gini               1         random         0.221946   
2             gini               4           best         0.263880   
3             gini               4         random         0.264757   
4             gini               7           best         0.321743   
5             gini               7         random         0.324348   
6             gini               8           best         0.352711   
7             gini               8         random         0.318168   
8             gini               9           best         0.381670   
9             gini               9         random         0.336725   
10            gini              10           best         0.415649   
11            gini              10         random         0.380267   
12         entropy               1           best         0.226203   
13         entropy  

In [96]:
model = clf.best_estimator_
print('Resultado Arbol de Decisión:' , model.score(X_train,Y_train)*100)

Resultado Arbol de Decisión: 41.34966630957688


In [97]:
n_nodes = model.tree_.node_count
children_left = model.tree_.children_left
children_right = model.tree_.children_right
feature = model.tree_.feature
threshold = model.tree_.threshold

node_depth = np.zeros(shape=n_nodes, dtype=np.int64)
is_leaves = np.zeros(shape=n_nodes, dtype=bool)
stack = [(0, -1)]  # seed is the root node id and its parent depth
while len(stack) > 0:
    node_id, parent_depth = stack.pop()
    node_depth[node_id] = parent_depth + 1

    # If we have a test node
    if (children_left[node_id] != children_right[node_id]):
        stack.append((children_left[node_id], parent_depth + 1))
        stack.append((children_right[node_id], parent_depth + 1))
    else:
        is_leaves[node_id] = True

print("The binary tree structure has %s nodes and has "
      "the following tree structure:"
      % n_nodes)
for i in range(n_nodes):
    if is_leaves[i]:
        print("%snode=%s leaf node." % (node_depth[i] * "\t", i))
    else:
        print("%snode=%s test node: go to node %s if X[:, %s] <= %s else to "
              "node %s."
              % (node_depth[i] * "\t",
                 i,
                 children_left[i],
                 feature[i],
                 threshold[i],
                 children_right[i],
                 ))
print()


The binary tree structure has 1979 nodes and has the following tree structure:
node=0 test node: go to node 1 if X[:, 6] <= 3.5 else to node 1018.
	node=1 test node: go to node 2 if X[:, 4] <= 3.5 else to node 513.
		node=2 test node: go to node 3 if X[:, 0] <= 3.5 else to node 258.
			node=3 test node: go to node 4 if X[:, 9] <= 3.5 else to node 131.
				node=4 test node: go to node 5 if X[:, 3] <= 5.5 else to node 68.
					node=5 test node: go to node 6 if X[:, 5] <= 6.5 else to node 37.
						node=6 test node: go to node 7 if X[:, 7] <= 6.5 else to node 22.
							node=7 test node: go to node 8 if X[:, 1] <= 7.5 else to node 15.
								node=8 test node: go to node 9 if X[:, 3] <= 3.5 else to node 12.
									node=9 test node: go to node 10 if X[:, 5] <= 3.5 else to node 11.
										node=10 leaf node.
										node=11 leaf node.
									node=12 test node: go to node 13 if X[:, 1] <= 4.5 else to node 14.
										node=13 leaf node.
										node=14 leaf node.
								node=15 t

In [98]:
######## NAIVE BAYES ##########
bayes = MultinomialNB()
val_cruzada = cross_validate(bayes, X_train, Y_train, cv = 3)
print('Resultado Naive Bayes: ', val_cruzada['test_score'])

Resultado Naive Bayes:  [0.22663106 0.22716602 0.225574  ]
